In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
#hide
from fastbook import *
from IPython.display import display,HTML

# Downloading the dataset

In [3]:
from fastai.text.all import *
path = untar_data(URLs.AMAZON_REVIEWS_POLARITY)

In [4]:
train = pd.read_csv(path/'train.csv')
test = pd.read_csv(path/'test.csv')

In [ ]:
len(train)

In [ ]:
train.shape

In [5]:
train.columns = ['label', 'review_title', 'text']
test.columns = ['label', 'review_title', 'text']
train.head()

,label,review_title,text
0,2,The best soundtrack ever to anything.,"I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny."
1,2,Amazing!,"This soundtrack is my favorite music of all time, hands down. The intense sadness of ""Prisoners of Fate"" (which means all the more if you've played the game) and the hope in ""A Distant Promise"" and ""Girl who Stole the Star"" have been an important inspiration to me personally throughout my teen years. The higher energy tracks like ""Chrono Cross ~ Time's Scar~"", ""Time of the Dreamwatch"", and ""Chronomantique"" (indefinably remeniscent of Chrono Trigger) are all absolutely superb as well.This soundtrack is amazing music, probably the best of this composer's work (I haven't heard the Xenogears s..."
2,2,Excellent Soundtrack,"I truly like this soundtrack and I enjoy video game music. I have played this game and most of the music on here I enjoy and it's truly relaxing and peaceful.On disk one. my favorites are Scars Of Time, Between Life and Death, Forest Of Illusion, Fortress of Ancient Dragons, Lost Fragment, and Drowned Valley.Disk Two: The Draggons, Galdorb - Home, Chronomantique, Prisoners of Fate, Gale, and my girlfriend likes ZelbessDisk Three: The best of the three. Garden Of God, Chronopolis, Fates, Jellyfish sea, Burning Orphange, Dragon's Prayer, Tower Of Stars, Dragon God, and Radical Dreamers - Uns..."
3,2,"Remember, Pull Your Jaw Off The Floor After Hearing it","If you've played the game, you know how divine the music is! Every single song tells a story of the game, it's that good! The greatest songs are without a doubt, Chrono Cross: Time's Scar, Magical Dreamers: The Wind, The Stars, and the Sea and Radical Dreamers: Unstolen Jewel. (Translation varies) This music is perfect if you ask me, the best it can be. Yasunori Mitsuda just poured his heart on and wrote it down on paper."
4,2,an absolute masterpiece,"I am quite sure any of you actually taking the time to read this have played the game at least once, and heard at least a few of the tracks here. And whether you were aware of it or not, Mitsuda's music contributed greatly to the mood of every single minute of the whole game.Composed of 3 CDs and quite a few songs (I haven't an exact count), all of which are heart-rendering and impressively remarkable, this soundtrack is one I assure you you will not forget. It has everything for every listener -- from fast-paced and energetic (Dancing the Tokage or Termina Home), to slower and more haunti..."


In [ ]:
test.head()

# Training a Text Classifier

In [ ]:
torch.cuda.empty_cache()

In [30]:
dls = TextDataLoaders.from_df(train[:100000], path=path, text_col='text', is_lm=True)
dls.show_batch(max_n=3)

/home/orjan/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,text,text_
0,"xxbos xxmaj the stitching came out along the belly after about 2 days of light to moderate play . xxmaj replaced it w / the xxmaj hedgehog which is still going strong after 4 months . xxbos xxmaj i 'm neutral to xxmaj mel xxmaj gibson . xxmaj he 's getting old . xxmaj the movie was n't acted all that great , but it did have some humor . xxmaj it","xxmaj the stitching came out along the belly after about 2 days of light to moderate play . xxmaj replaced it w / the xxmaj hedgehog which is still going strong after 4 months . xxbos xxmaj i 'm neutral to xxmaj mel xxmaj gibson . xxmaj he 's getting old . xxmaj the movie was n't acted all that great , but it did have some humor . xxmaj it 's"
1,"mercenary xxmaj doc . xxbos xxmaj the entries in this dictionary are great and informative , but it just seems like it could have had even more in it ( plus , there are a few entries that probably did n't need to be in there such as xxmaj karl xxmaj popper , a great philosopher of science and all , but how much does he really affect critical theory ? )","xxmaj doc . xxbos xxmaj the entries in this dictionary are great and informative , but it just seems like it could have had even more in it ( plus , there are a few entries that probably did n't need to be in there such as xxmaj karl xxmaj popper , a great philosopher of science and all , but how much does he really affect critical theory ? ) ."
2,"of the xxmaj triffids based on another book by xxmaj john xxmaj wyndham . xxbos xxmaj this is , for me , their finest album / xxup cd . xxmaj without question , it 's a great collection of songs , from top to bottom - period . xxmaj it also happens to be their last with the great xxmaj thom xxmaj bell and the late xxmaj linda xxmaj creed . xxmaj","the xxmaj triffids based on another book by xxmaj john xxmaj wyndham . xxbos xxmaj this is , for me , their finest album / xxup cd . xxmaj without question , it 's a great collection of songs , from top to bottom - period . xxmaj it also happens to be their last with the great xxmaj thom xxmaj bell and the late xxmaj linda xxmaj creed . xxmaj my"


In [31]:
learn = language_model_learner(
    dls, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [32]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.039893,3.931554,0.287850,50.986130,04:04


In [33]:
learn.save('1epoch')

Path('/home/orjan/.fastai/data/amazon_review_polarity_csv/models/1epoch.pth')

In [34]:
learn = learn.load('1epoch')

In [35]:
learn.unfreeze()
learn.fit_one_cycle(1,2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.650481,3.720498,0.313502,41.284943,04:23


In [42]:
learn.save_encoder('finetuned')

# Creating the classifier

In [37]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_df('text', vocab=dls.vocab),CategoryBlock),
    get_x=ColReader('text'), get_y=ColReader('label'), 
    splitter=RandomSplitter(0.1)
).dataloaders(test[:100000])

/home/orjan/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [38]:
dls_clas.show_batch(max_n=2)

,text,category
0,"xxbos i xxup got xxup this "" album "" xxup for xxup my xxup thirteenth xxup birthday , xxup in xxunk xxup remember xxup the xxup album xxup came xxup with a xxup poster xxup and i xxup had xxup it xxup hanging xxup up xxup in xxup my xxup room xxup for xxup years ) . i xxup have xxup been xxup listening xxup to xxup it xxup ever xxup since , i xxup will xxup never , xxup ever xxup get xxup tired xxup of xxup hearing xxup this xxup one . xxup it xxup may xxup well xxup be xxup the xxup greatest ( and xxup certainly xxup one xxup of xxup the xxup first ) "" live "" xxup rock xxup and xxup roll xxup albums xxup of xxup all xxup time . xxup you xxup need xxup to xxup turn xxup the xxup volume xxup",2
1,xxbos i would not even give this a star if i did not have to . i bought this movie at my local xxup xxunk store and it was xxunk and i got thinking it was going to be good but it is the worst movie ans i do nt think it should have been made on blue ray let alone blue ray i think xxunk ever buys this movie is basically flushing their money down the drain . people let me help you save your money and say xxup do nt xxup buy xxup this xxup movie xxup it xxup is a xxup complete xxup waste xxup of xxup money xxup xxunk xxup it xxup was a xxup waste xxup of xxup my xxup time xxup watching xxup it xxup after i xxup watched xxup it i xxup threw xxup it xxup in xxup the xxup fire xxup because,1


In [39]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [43]:
learn = learn.load_encoder('finetuned')

In [44]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.440751,0.353552,0.843200,00:40


In [45]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.302367,0.257824,0.896800,00:48


In [46]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.239882,0.218221,0.911700,01:07


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.214464,0.211961,0.914700,01:33
